In [1]:
import os
import glob
import json
import re

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [2]:
laws_dir = "laws"  # JSON 파일들이 있는 디렉토리
json_files = glob.glob(os.path.join(laws_dir, "*.json"))

json_files

['laws/형의집행및수용자의처우에관한법률시행령.json',
 'laws/자동차손해배상보장법.json',
 'laws/형법.json',
 'laws/자동차손해배상보장법시행규칙.json',
 'laws/교통사고처리특례법.json',
 'laws/도로교통법시행령.json',
 'laws/자동차손해배상보장법시행령.json',
 'laws/교통사고처리특례법시행령.json',
 'laws/도로교통법.json',
 'laws/형의집행및수용자의처우에관한법률.json',
 'laws/민법.json',
 'laws/특정범죄가중처벌등에관한법률.json',
 'laws/도로교통법시행규칙.json']

In [3]:
def clean_text(text):
    """
    text가 문자열이 아닐 경우(예: list, dict) 등을 처리해주는 버전
    """
    if isinstance(text, list):
        # 리스트이면 각 항목을 문자열로 변환 후 공백으로 join
        text = " ".join(str(t) for t in text)
    elif not isinstance(text, str):
        text = str(text)
    
    text = text.replace("\r", " ").replace("\t", " ")
    text = re.sub(r"\s+", " ", text)
    text = text.strip()
    return text

def extract_text_from_law_json(json_data: dict) -> list:
    results = []
    law = json_data.get("법령", {})

    # (1) 법령명
    law_name = law.get("기본정보", {}).get("법령명_한글", "")
    results.append(f"[{clean_text(law_name)}]")

    # (2) 부칙
    sup_provisions = law.get("부칙", {}).get("부칙단위", [])
    for sup_provision in sup_provisions:
        content = sup_provision.get("부칙내용", [])
        for paragraph_list in content:  # 2차원 리스트
            cleaned_line = []
            for line in paragraph_list:
                line = clean_text(line)
                if line:
                    cleaned_line.append(line)
            merged = "".join(cleaned_line)
            if merged:
                results.append(merged)

    # (3) 조문
    provisions = law.get("조문", {}).get("조문단위", [])
    for provision in provisions:
        article_text = provision.get("조문내용", "")
        article_text = clean_text(article_text)
        if article_text:
            results.append(article_text)

        if "항" in provision:
            if isinstance(provision["항"], dict):
                # 항이 dict
                ho_list = provision["항"].get("호", [])
                if isinstance(ho_list, list):
                    for ho_item in ho_list:
                        ho_text = ho_item.get("호내용", "")
                        ho_text = clean_text(ho_text)
                        if ho_text:
                            results.append(ho_text)
            elif isinstance(provision["항"], list):
                for paragraph_item in provision["항"]:
                    if isinstance(paragraph_item, dict):
                        para_text = paragraph_item.get("항내용", "")
                        para_text = clean_text(para_text)
                        if para_text:
                            results.append(para_text)

                        if "호" in paragraph_item:
                            ho_list = paragraph_item["호"]
                            if isinstance(ho_list, list):
                                for ho_item in ho_list:
                                    ho_text = ho_item.get("호내용", "")
                                    ho_text = clean_text(ho_text)
                                    if ho_text:
                                        results.append(ho_text)

    # (4) 별표
    if "별표" in law:
        # law["별표"] => {"별표단위": [ {...}, {...} ]}
        annex_container = law["별표"]
        if isinstance(annex_container, dict):
            # "별표단위" 리스트를 꺼냄
            table_list = annex_container.get("별표단위", [])
            if isinstance(table_list, list):
                for table_item in table_list:
                    if isinstance(table_item, dict):
                        _extract_table_item(table_item, results)

    return results


def _extract_table_item(table_dict: dict, results: list):
    """
    '별표단위'의 한 요소(별표번호, 별표제목, 별표내용 등)를 줄 단위로 전처리하여 results에 추가
    """
    # 1) 별표번호
    table_num = table_dict.get("별표번호", "")
    table_num = clean_text(table_num)
    
    # 2) 별표제목
    table_title = table_dict.get("별표제목", "")
    table_title = clean_text(table_title)
    
    # 결과에 별표번호 / 별표제목 저장
    if table_num or table_title:
        results.append(f"(별표번호) {table_num} (별표제목) {table_title}")

    # 3) 별표내용
    table_contents = table_dict.get("별표내용", [])
    # 별표내용이 2차원 리스트라고 가정
    if isinstance(table_contents, list):
        for paragraph_list in table_contents:
            if isinstance(paragraph_list, list):
                for line in paragraph_list:
                    if isinstance(line, str):
                        txt = clean_text(line)
                        if txt:
                            results.append(txt)


def chunk_text(text_list: list, max_chunk_size: int = 500) -> list:
    chunks = []
    for text in text_list:
        if len(text) <= max_chunk_size:
            chunks.append(text)
        else:
            start = 0
            while start < len(text):
                end = start + max_chunk_size
                chunks.append(text[start:end])
                start = end
    return chunks

In [4]:
FILE_PATH = 'laws/도로교통법시행령.json'
with open(FILE_PATH, "r", encoding='utf-8') as f:
    json_data = json.load(f)
json_data

{'법령': {'개정문': {'개정문내용': [['⊙대통령령 제35163호(2024.12.31)',
     '여객자동차 운수사업법 시행령 일부개정령',
     '[본문 생략]',
     '        부칙',
     '제1조(시행일) 이 영은 공포한 날부터 시행한다.',
     '제2조(다른 법령의 개정) 도로교통법 시행령 일부를 다음과 같이 개정한다.',
     '  제31조제4호 중 "「여객자동차 운수사업법 시행령」 제3조제2호가목 단서"를 "「여객자동차 운수사업법 시행령」 제3조의2제1항제1호"로 한다.']]},
  '법령키': '0033952024123135163',
  '별표': {'별표단위': [{'별표제목': '전용차로의 종류와 전용차로로 통행할 수 있는 차(제9조제1항 관련)',
     '별표PDF파일명': 'law0033952024123135163KC_000100E.pdf',
     '별표HWP파일명': 'law0033952024123135163KC_000100E.hwp',
     '별표번호': '0001',
     '별표서식PDF파일링크': '/LSW/flDownload.do?flSeq=147605327',
     '별표키': '000100E',
     '별표내용': [['■ 도로교통법 시행령 [별표 1] <개정 2020. 12. 31.>',
       '                                                                                      ',
       '  전용차로의 종류와 전용차로로 통행할 수 있는 차(제9조제1항 관련)                     ',
       '┏━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓',
       '┃전용차로의  │통행할 수 있는 차                                                   ┃',
       '┃종류        ├────

In [5]:
extracted_texts = extract_text_from_law_json(json_data)
extracted_texts

['[도로교통법 시행령]',
 '부칙 <제19493호, 2006.5.30>제1조 (시행일) 이 영은 2006년 6월 1일부터 시행한다.제2조 (종전의 면허 등에 관한 경과조치) 이 영 시행 당시 종전의 규정에 의한 행정기관의 행위 또는 행정기관에 대한 행위는 그에 해당하는 이 영에 의한 행정기관의 행위 또는 행정기관에 대한 행위로 본다.제3조 (과태료 및 범칙행위의 처리에 관한 경과조치) 이 영 시행 전의 행위에 대한 과태료의 부과ㆍ징수 및 범칙행위의 처리에 있어서는 종전의 규정에 의한다.제4조 (다른 법령의 개정) ①경찰공무원임용령 일부를 다음과 같이 개정한다.제39조제4항중 "제68조제2항제1호"를 "제80조제2항제1호"로 한다.②과세자료의제출및관리에관한법률시행령 일부를 다음과 같이 개정한다.별표 일련번호 24의 과세자료명란중 "제70조의2"를 "제99조"로 한다.③국가채권관리법시행령 일부를 다음과 같이 개정한다.제2조제1호중 "제84조"를 "제163조"로 한다.④대중교통의 육성 및 이용촉진에 관한 법률 시행령 일부를 다음과 같이 개정한다.제11조제2호중 "제13조의2"를 "제15조"로 한다.⑤보험업법 시행령 일부를 다음과 같이 개정한다.제80조제1항제13호중 "제42조의5제1항, 제49조의2제2항 및 별표 1의3 제9호"를 "제63조제1항, 제67조제2항 및 별표 5 제9호"로 한다.⑥소방공무원임용령 일부를 다음과 같이 개정한다.제43조제4항중 "제68조제2항제1호"를 "제80조제2항제1호"로 한다.⑦옥외광고물 등 관리법 시행령 일부를 다음과 같이 개정한다.제29조제6호중 "제2조제10호"를 "제2조제13호"로 한다.⑧응급의료에관한법률시행령 일부를 다음과 같이 개정한다.제8조제4호중 "제5조"를 "제5조제1항"으로 한다.⑨자동차관리법시행령 일부를 다음과 같이 개정한다.제7조제1항제12호중 "제70조의2 및 제71조의2"를 "제99조 및 제104조"로 하고, "제42조의5제2항 및 제49조의2제3항"을 "제63조제2항 및 제67조제3항"으로 한다.⑩자동차손